In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, silhouette_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
from sklearn.preprocessing import StandardScaler, normalize
import matplotlib as mpl

---
# Training models on 3 small samples of cluster pairs

In [ ]:
sample1=pd.read_parquet("old_new.parquet")
sample2=pd.read_parquet("hiphop_country.parquet")
sample3=pd.read_parquet("sad_bright.parquet")

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=13)

In [ ]:
# Old vs New
X_audio1=sample1[sample1.columns[3:]].values
y_audio1=sample1["Cluster"].values
X_lyrics1=np.vstack(old_new["lyrics_embeddings"].values)
y_lyrics1=old_new["Cluster"].values

In [ ]:
y_pred_audio1 = kmeans.fit_predict(X_audio1)
ari_audio1 = adjusted_rand_score(y_audio1, y_pred_audio1)
print("Adjusted Rand Index (ARI) for audio embeddings:", ari_audio1)
y_pred_lyrics1=kmeans.fit_predict(X_lyrics1)
ari_lyrics1 = adjusted_rand_score(y_lyrics1, y_pred_lyrics1)
print("Adjusted Rand Index (ARI) for lyric embeddings:", ari_lyrics1)

In [ ]:
## Cluters generated by K-means are not inherent so we have to map it to the true labels to understand the performance of the model
def map_clusters(true_labels, predicted_labels):
    """
    Matches predicted cluster labels to true labels using the Hungarian algorithm.
    Returns a mapping dictionary where keys are predicted labels and values are the corresponding true labels.
    """
    # Compute the contingency matrix (rows: true labels, columns: predicted labels)
    cm = confusion_matrix(true_labels, predicted_labels)
    
    # Negate the matrix to convert maximization into minimization
    cost_matrix = -cm
    
    # Apply the Hungarian algorithm (linear_sum_assignment)
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    
    # Build the mapping: for each predicted label (col), assign the corresponding true label (row)
    mapping = {pred: true for true, pred in zip(row_ind, col_ind)}
    return mapping

In [ ]:
#Performance of audio embeddings on old-new cluster pair
true_labels_audio1 = np.array(y_audio1)       
predicted_labels_audio1 = np.array(y_pred_audio1)
mapping = map_clusters(true_labels_audio1, predicted_labels_audio1)
print("Mapping of predicted to true labels:", mapping)

# Remap predicted labels based on the mapping:
mapped_predicted_labels = np.array([mapping.get(label, label) for label in predicted_labels_audio1])
print("Mapped predicted labels:", mapped_predicted_labels)

# Create confusion matrix with remapped labels
cm_mapped = confusion_matrix(true_labels_audio1, mapped_predicted_labels)
cm_df = pd.DataFrame(cm_mapped, 
                     index=[f"True {i}" for i in sorted(np.unique(true_labels_audio1))],
                     columns=[f"Pred {i}" for i in sorted(np.unique(mapped_predicted_labels))])
print("\nConfusion Matrix (after mapping):")
print(cm_df)

# Print classification report
report = classification_report(true_labels_audio1, mapped_predicted_labels)
print("\nClassification Report:")
print(report)   

In [ ]:
#Performance of lyrics embeddings on old-new cluster pair
true_labels_lyrics1 = np.array(y_lyrics1)       
predicted_labels_lyrics1 = np.array(y_pred_lyrics1)
mapping = map_clusters(true_labels_lyrics1, predicted_labels_lyrics1)
print("Mapping of predicted to true labels:", mapping)

# Remap predicted labels based on the mapping:
mapped_predicted_labels = np.array([mapping.get(label, label) for label in predicted_labels_lyrics1])
print("Mapped predicted labels:", mapped_predicted_labels)

# Create confusion matrix with remapped labels
cm_mapped = confusion_matrix(true_labels_lyrics1, mapped_predicted_labels)
cm_df = pd.DataFrame(cm_mapped, 
                     index=[f"True {i}" for i in sorted(np.unique(true_labels_lyrics1))],
                     columns=[f"Pred {i}" for i in sorted(np.unique(mapped_predicted_labels))])
print("\nConfusion Matrix (after mapping):")
print(cm_df)

# Print classification report
report = classification_report(true_labels_lyrics1, mapped_predicted_labels)
print("\nClassification Report:")
print(report)

In [ ]:
# Hiphop vs Pop Rock
X_audio2=sample2[sample2.columns[3:]].values
y_audio2=sample2["Cluster"].values
X_lyrics2=np.vstack(hiphop_nonhiphop["lyrics_embeddings"].values)
y_lyrics2=hiphop_nonhiphop["Cluster"].values

In [ ]:
y_pred_audio2 = kmeans.fit_predict(X_audio2)
ari_audio2 = adjusted_rand_score(y_audio2, y_pred_audio2)
print("Adjusted Rand Index (ARI) for audio embeddings:", ari_audio2)
y_pred_lyrics2=kmeans.fit_predict(X_lyrics2)
ari_lyrics2 = adjusted_rand_score(y_lyrics2, y_pred_lyrics2)
print("Adjusted Rand Index (ARI) for lyric embeddings:", ari_lyrics2)

In [ ]:
#Confusion Matrix for audio embeddings hiphop vs country/pop
true_labels_audio2 = np.array(y_audio2)       
predicted_labels_audio2 = np.array(y_pred_audio2)   
mapping = map_clusters(true_labels_audio2, predicted_labels_audio2)
print("Mapping of predicted to true labels:", mapping)

# Remap predicted labels based on the mapping:
mapped_predicted_labels = np.array([mapping.get(label, label) for label in predicted_labels_audio2])
print("Mapped predicted labels:", mapped_predicted_labels)

# Create confusion matrix with remapped labels
cm_mapped = confusion_matrix(true_labels_audio2, mapped_predicted_labels)
cm_df = pd.DataFrame(cm_mapped, 
                     index=[f"True {i}" for i in sorted(np.unique(true_labels_audio2))],
                     columns=[f"Pred {i}" for i in sorted(np.unique(mapped_predicted_labels))])
print("\nConfusion Matrix (after mapping):")
print(cm_df)

# Print classification report
report = classification_report(true_labels_audio2, mapped_predicted_labels)
print("\nClassification Report:")
print(report)

In [ ]:
### Matrix for lyrics embeddings hiphop vs countr/pop
true_labels_lyrics2 = np.array(y_lyrics2)       
predicted_labels_lyrics2 = np.array(y_pred_lyrics2)
mapping = map_clusters(true_labels_lyrics2, predicted_labels_lyrics2)
print("Mapping of predicted to true labels:", mapping)

# Remap predicted labels based on the mapping:
mapped_predicted_labels = np.array([mapping.get(label, label) for label in predicted_labels_lyrics2])
print("Mapped predicted labels:", mapped_predicted_labels)

# Create confusion matrix with remapped labels
cm_mapped = confusion_matrix(true_labels_lyrics2, mapped_predicted_labels)
cm_df = pd.DataFrame(cm_mapped, 
                     index=[f"True {i}" for i in sorted(np.unique(true_labels_lyrics2))],
                     columns=[f"Pred {i}" for i in sorted(np.unique(mapped_predicted_labels))])
print("\nConfusion Matrix (after mapping):")
print(cm_df)

# Print classification report
report = classification_report(true_labels_lyrics2, mapped_predicted_labels)
print("\nClassification Report:")
print(report)

In [ ]:
##Sad vs Bright
X_audio3=sample3[sample3.columns[3:]].values
y_audio3=sample3["Cluster"].values
X_lyrics3=np.vstack(sad_bright["lyrics_embeddings"].values)
y_lyrics3=sad_bright["Cluster"].values

In [ ]:
y_pred_audio3 = kmeans.fit_predict(X_audio3)
ari_audio3 = adjusted_rand_score(y_audio3, y_pred_audio3)
print("Adjusted Rand Index (ARI) for audio embeddings:", ari_audio3)
y_pred_lyrics3=kmeans.fit_predict(X_lyrics3)
ari_lyrics3 = adjusted_rand_score(y_lyrics3, y_pred_lyrics3)
print("Adjusted Rand Index (ARI) for lyric embeddings:", ari_lyrics3)

---
## Scale up model

In [ ]:
successfully_processed_sample=pd.read_parquet("processed_sample.parquet")
# Filter out those we weren't able to extract lyrics
with_lyrics=successfully_processed_sample[~successfully_processed_sample['lyrics_embeddings'].apply(lambda arr: all(x == 0 for x in arr))]

In [ ]:
## Since the BERT-based LLM generates 768-dimensional embeddings while the audio embeddings are 34-dimensional, we need to brigh them down to the same dimension before applying weights and combining them
# Preprocess and scale each modality.
X_audio = np.vstack(with_lyrics[with_lyrics.columns[2:-3]].values)  # shape: (n_samples, 34)
X_lyrics = np.vstack(with_lyrics[with_lyrics.columns[-1]].values)   # shape: (n_samples, 768)

scaler_audio = StandardScaler()
scaler_lyrics = StandardScaler()
X_audio_scaled = scaler_audio.fit_transform(X_audio)
X_lyrics_scaled = scaler_lyrics.fit_transform(X_lyrics)

X_audio_norm = normalize(X_audio_scaled, norm='l2')
X_lyrics_norm = normalize(X_lyrics_scaled, norm='l2')
# Step 2: Project each modality to a common latent space.
common_dim = 34
pca_audio = PCA(n_components=common_dim, random_state=13)
pca_lyrics = PCA(n_components=common_dim, random_state=13)
X_audio_proj = pca_audio.fit_transform(X_audio_norm)    # shape: (n_samples, common_dim)
X_lyrics_proj = pca_lyrics.fit_transform(X_lyrics_norm)    # shape: (n_samples, common_dim)
# Step 3: Fuse the embeddings via weighted sum.
# Set weights. If you want clustering solely based on lyrics, set lyrics_weight = 1 and audio_weight = 0.
lyrics_weight = 0.4   # Predefine this from models' performances on the small samples
audio_weight = 1.0 - lyrics_weight

def fuse_embeddings(audio_proj, lyric_proj, w_audio, w_lyrics):
    return w_audio * audio_proj + w_lyrics * lyric_proj

X_fused = fuse_embeddings(X_audio_proj, X_lyrics_proj, audio_weight, lyrics_weight)
print("Fused embedding shape:", X_fused.shape)
with_lyrics['combined_embedding'] = list(X_fused)
# Step 4: Cluster using K-means on the fused embeddings.
best_k = None
best_score = -1
best_labels = None
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=13, n_init=20)
    labels = kmeans.fit_predict(X_fused)
    score = silhouette_score(X_fused, labels)
    print(f"k = {k}, silhouette score = {score:.4f}")
    if score > best_score:
        best_score = score
        best_k = k
        best_labels = labels

print(f"\nBest k: {best_k} with silhouette score: {best_score:.4f}")
# Step 5: PCA visualization (optional, for 2D visualization)
plt.figure(figsize=(8,6))
# Define boundaries to force two discrete bins: anything below 0.5 maps to 0 and above maps to 1.
boundaries = [-0.5, 0.5, 1.5]
norm = mpl.colors.BoundaryNorm(boundaries, ncolors=256, clip=True)

# Create the scatter plot using the norm.
scatter = plt.scatter(X_2d[:, 0], X_2d[:, 1], c=best_labels, cmap='viridis', norm=norm, alpha=0.7)
plt.title("Clusters visualized in 2-dimensional space")
plt.xlabel("The direction in which the data has the largest spread (variance)")
plt.ylabel("The next direction of greatest spread, but orthogonal to PC1")

# Create a colorbar with the discrete boundaries.
cbar = plt.colorbar(scatter, boundaries=boundaries, ticks=[0, 1])
cbar.ax.set_yticklabels(['Cluster 1', 'Cluster 2'])
plt.show()

# Optionally, compute overall silhouette score
overall_silhouette = silhouette_score(X_fused, best_labels)
print(f"Overall silhouette score: {overall_silhouette:.4f}")

# Optionally, compute overall silhouette score
overall_silhouette = silhouette_score(X_fused, best_labels)
print(f"Overall silhouette score: {overall_silhouette:.4f}")

---
## Compute cosine similarity within clusters and generate recommendations


In [ ]:
def compute_intra_cluster_similarity(df, cluster_col='cluster_ensemble', embedding_col='combined_embedding'):
    """
    Compute the average cosine similarity for songs within each cluster and overall.
    
    Parameters:
      df: DataFrame containing cluster labels and embeddings.
      cluster_col: Name of the column with cluster labels.
      embedding_col: Name of the column with the fused embedding (as a NumPy array).
      
    Returns:
      cluster_similarity: Dictionary mapping each cluster label to its average cosine similarity.
      overall_avg_similarity: Average cosine similarity across clusters.
    """
    cluster_similarity = {}
    # Loop over each unique cluster.
    for cl in df[cluster_col].unique():
        # Get all embeddings for this cluster.
        cluster_df = df[df[cluster_col] == cl]
        embeddings = np.vstack(cluster_df[embedding_col].values)
        n_samples = embeddings.shape[0]
        if n_samples < 2:
            # Cannot compute similarity for a single song; skip or set to NaN.
            cluster_similarity[cl] = np.nan
            continue
        # Compute cosine similarity matrix.
        sim_matrix = cosine_similarity(embeddings)
        # Exclude the diagonal (self-similarity = 1) by subtracting n_samples.
        # Then average over all off-diagonal elements.
        total_sim = np.sum(sim_matrix) - n_samples  # subtract diagonal ones.
        num_pairs = n_samples * (n_samples - 1)
        avg_sim = total_sim / num_pairs
        cluster_similarity[cl] = avg_sim

    # Compute overall average (ignoring clusters with NaN values)
    valid_sims = [sim for sim in cluster_similarity.values() if not np.isnan(sim)]
    overall_avg_similarity = np.mean(valid_sims) if valid_sims else np.nan
    return cluster_similarity, overall_avg_similarity

In [ ]:
cluster_sim, overall_sim = compute_intra_cluster_similarity(with_lyrics)
print("Intra-cluster cosine similarity for each cluster:")
for cl, sim in cluster_sim.items():
    print(f"Cluster {cl}: {sim:.4f}")
print(f"Overall average intra-cluster cosine similarity: {overall_sim:.4f}")

In [ ]:
def recommend_top_10(query_index, df):
    """
    Given a query song index, computes cosine similarity between its combined embedding
    (stored in df['combined_embedding']) and all songs, then returns:
      - A DataFrame with the top 10 most similar songs (excluding the query),
      - The overall similarity (average cosine similarity of these top 10 songs),
      - The similarity scores for each recommended song.
    
    Parameters:
      query_index (int): Index of the query song in df.
      df (DataFrame): DataFrame with at least a 'combined_embedding' column containing
                      the fused embedding (as a NumPy array) and metadata columns like 'title' and 'artist'.
    
    Returns:
      recommendations (DataFrame): Top 10 similar songs with their metadata.
      overall_similarity (float): The average cosine similarity of the top 10 songs.
      top_similarities (np.array): The cosine similarity scores for the top 10 songs.
    """
    # Extract the query song's embedding and reshape to 2D.
    query_embedding = df.loc[query_index, 'combined_embedding'].reshape(1, -1)
    
    # Stack all combined embeddings into a 2D array.
    all_embeddings = np.vstack(df['combined_embedding'].values)
    
    # Compute cosine similarity between query and all songs.
    similarities = cosine_similarity(query_embedding, all_embeddings)[0]
    
    # Sort indices by similarity in descending order.
    sorted_indices = np.argsort(similarities)[::-1]
    
    # Exclude the query song itself.
    sorted_indices = sorted_indices[sorted_indices != query_index]
    
    # Select the top 10 most similar song indices.
    top_indices = sorted_indices[:10]
    
    # Get the similarity scores for these top songs.
    top_similarities = similarities[top_indices]
    
    # Compute overall similarity (average of the top 10 similarities).
    overall_similarity = np.mean(top_similarities)
    
    # Retrieve metadata (e.g., title, artist) for recommended songs.
    recommendations = df.iloc[top_indices][['title', 'artist']]
    
    return recommendations, overall_similarity, top_similarities

In [ ]:
import random
with_lyrics=with_lyrics.reset_index(drop=True)
for i in range(0,10):
    query_idx=random.randint(0, len(with_lyrics))
    recs, overall_sim, sim_scores = recommend_top_10(query_idx, with_lyrics)
    print("Top 10 recommendations for song index", query_idx)
    print(recs)
    print("Overall average similarity:", overall_sim)
    print("Similarity scores:", sim_scores)
